In [3]:
!pip3 install -r ./requirements.txt -q

You should consider upgrading via the '/Library/Frameworks/Python.framework/Versions/3.10/bin/python3.10 -m pip install --upgrade pip' command.


In [26]:
#pip install openai

In [4]:
pip install python-dotenv


[notice] A new release of pip is available: 23.3.1 -> 23.3.2
[notice] To update, run: /opt/homebrew/Cellar/jupyterlab/4.0.9_2/libexec/bin/python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


### Python-dotenv

In [4]:
import os
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv(),override=True)
os.environ.get('PINECONE_API_KEY')

'e4f1b2bb-454a-468f-9056-f7bcb8764b06'

### LLM Models (Wrappers):GPT-3

In [16]:
from langchain.llms import OpenAI

In [28]:
llm = OpenAI(model_name='text-davinci-003',temperature=0.7,max_tokens=512)
print(llm)

OpenAI
Params: {'model_name': 'text-davinci-003', 'temperature': 0.7, 'top_p': 1, 'frequency_penalty': 0, 'presence_penalty': 0, 'n': 1, 'logit_bias': {}, 'max_tokens': 512}


In [29]:
output = llm('explain smart contract in one sentence')
print(output)



A smart contract is a self-executing digital agreement stored on a blockchain that is designed to automatically enforce the terms of a contract.


### ChatModels: GPT-3.5-Turbo and GPT-4

In [2]:
from langchain.schema import(AIMessage,HumanMessage,SystemMessage)
from langchain.chat_models import ChatOpenAI

In [6]:
chat = ChatOpenAI(model_name='gpt-3.5-turbo', temperature=0.5,max_tokens=1024)
messages=[
    SystemMessage(content='You are a physicist and respond only in German.'),
    HumanMessage(content='explain quantum mechanics in one sentence')
]
output = chat(messages)

In [7]:
print(output)

content='Quantenmechanik beschreibt das Verhalten von Teilchen auf subatomarer Ebene, indem sie sowohl Wellen- als auch Teilcheneigenschaften besitzen.'


### Prompt Templates